In [1]:
import torch
import pandas as pd
import numpy as np
import os

# Set project directory
project_dir = r"C:\Users\aknin\Desktop\MLproject-main"
model_path = os.path.join(project_dir, "soc_lstm_model_new.pth")
predict_data_path = os.path.join(project_dir, "predictSoc.csv")  # Adjust path if different
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define SoCLSTM class (same as used in training)
class SoCLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SoCLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=x.dtype, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=x.dtype, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Define feature columns from your data
FEATURE_COLS = ['Voltage [V]', 'Current [A]', 'Temperature [degC]', 'Capacity [Ah]', 'Cumulative_Capacity_Ah']

# Load the scaler (refit here for simplicity; in practice, save and load it)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Preprocess new data function
def preprocess_new_data(file_path):
    df = pd.read_csv(file_path)
    if 'Power [W]' not in df.columns:
        df['Power [W]'] = df['Voltage [V]'] * df['Current [A]']
    if 'Cumulative_Capacity_Ah' not in df.columns:
        df['Cumulative_Capacity_Ah'] = (df['Current [A]'] * df['Time [s]'].diff().fillna(0) / 3600).cumsum()
    if 'Time [s]' not in df.columns:
        df['Time [s]'] = range(len(df))  # Assume sequential data with 1s intervals
    
    # Normalize features using the same scaler
    df[FEATURE_COLS] = scaler.fit_transform(df[FEATURE_COLS])
    features = df[FEATURE_COLS].values.astype(np.float32)
    SEQUENCE_LENGTH = 20
    sequences = []
    for i in range(len(features) - SEQUENCE_LENGTH + 1):
        sequences.append(features[i:i + SEQUENCE_LENGTH])
    sequences = torch.tensor(sequences, dtype=torch.float32).to(device)
    return sequences, df

# Load and test the model
best_hyperparams = {'hidden_size': 10, 'num_layers': 2, 'learning_rate': 0.011028140237963465}
loaded_model = SoCLSTM(input_size=len(FEATURE_COLS), hidden_size=best_hyperparams['hidden_size'], num_layers=best_hyperparams['num_layers']).to(device)
loaded_model.load_state_dict(torch.load(model_path, map_location=device)['model_state_dict'])
loaded_model.eval()

# Predict SOC function
def predict_soc(model, sequences, device=device):
    model.eval()
    with torch.no_grad():
        predictions = []
        for seq in sequences:
            seq = seq.unsqueeze(0).to(device)
            output = model(seq)
            predictions.append(output.cpu().item())
        return predictions

# Check if predictSoc.csv exists, otherwise simulate it
if not os.path.exists(predict_data_path):
    print(f"predictSoc.csv not found at {predict_data_path}. Creating a simulated dataset...")
    data = {
        'Voltage [V]': [4.17 + i * -0.001 for i in range(100)],
        'Current [A]': [-0.15 + i * 0.0005 for i in range(100)],
        'Temperature [degC]': [23.98 + i * 0.001 for i in range(100)],
        'Capacity [Ah]': [-0.005 + i * -0.0001 for i in range(100)],
        'Cumulative_Capacity_Ah': [0 + i * -0.00025 for i in range(100)],
        'Time [s]': [i for i in range(100)]
    }
    df = pd.DataFrame(data)
    df.to_csv(predict_data_path, index=False)
    print(f"Simulated predictSoc.csv created at {predict_data_path}")

# Preprocess and predict
new_sequences, df = preprocess_new_data(predict_data_path)
predictions = predict_soc(loaded_model, new_sequences)
print(f"Number of predictions: {len(predictions)}")
print(f"Sample predictions: {predictions[:5]}")

Number of predictions: 81
Sample predictions: [0.9550353288650513, 0.943553626537323, 0.9326677918434143, 0.9223183393478394, 0.9123696088790894]


C:\Users\aknin\AppData\Local\Temp\ipykernel_11356\2896896181.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  sequences = torch.tensor(sequences, dtype=torch.float32).to(device)
